In [22]:
import pandas as pd
#df=pd.read_csv('https://cloud.minsa.gob.pe/s/ZgXoXqK2KLjRLxD/download', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_VACUNACION', 'DOSIS', 'DEPARTAMENTO', 'FABRICANTE'],parse_dates=['FECHA_VACUNACION'])
df=pd.read_csv('vacunas_covid.csv', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_VACUNACION', 'DOSIS', 'DEPARTAMENTO', 'FABRICANTE'], parse_dates=['FECHA_VACUNACION'])
fecha_corte=df['FECHA_CORTE'].drop_duplicates().set_axis(['fecha_corte'])
#fecha_corte.to_json("../resultados/fecha_corte.json")

fecha_corte    20210509
Name: FECHA_CORTE, dtype: int64

In [2]:
#DIARIO DOSIS 1 Y DOSIS 2
df_ambas_dosis=df[['FECHA_VACUNACION','DOSIS','SEXO']].groupby(['FECHA_VACUNACION','DOSIS']).count()
df_ambas_dosis=df_ambas_dosis.reset_index()
df_ambas_dosis=df_ambas_dosis.pivot(index='FECHA_VACUNACION', columns='DOSIS', values='SEXO')
df_ambas_dosis=df_ambas_dosis.rename_axis(None, axis=1)
df_ambas_dosis.columns=['DOSIS1','DOSIS2']
df_ambas_dosis=df_ambas_dosis.fillna(0).astype('int')
df_ambas_dosis

,DOSIS1,DOSIS2
FECHA_VACUNACION,,
2021-02-09,3842,0
2021-02-10,16595,0
2021-02-11,33848,0
2021-02-12,32341,0
2021-02-13,18899,0
...,...,...
2021-05-05,17678,3609
2021-05-06,31650,1539
2021-05-07,24364,9778


In [3]:
#ACUMULADO DOSIS 1 Y DOSIS 2
df_ambas_dosis_cum=df_ambas_dosis.cumsum()
df_ambas_dosis_cum

,DOSIS1,DOSIS2
FECHA_VACUNACION,,
2021-02-09,3842,0
2021-02-10,20437,0
2021-02-11,54285,0
2021-02-12,86626,0
2021-02-13,105525,0
...,...,...
2021-05-05,1279294,664983
2021-05-06,1310944,666522
2021-05-07,1335308,676300


In [4]:
col_poblacion=[426806,
1180638,
430736,
1497438,
668213,
1453711,
1129854,
1357075,
365317,
760267,
975182,
1361467,
2016771,
1310785,
10628470,
1027559,
173811,
192740,
271904,
2047954,
1237997,
899648,
370974,
251521,
589110]


In [5]:
# ACUMULADO POR DEPARTAMENTO (DOSIS 2 - VACUNACION COMPLETA)
df_ambas_dosis_departamento=df[['DEPARTAMENTO','DOSIS','SEXO']].groupby(['DEPARTAMENTO', 'DOSIS']).count()
df_ambas_dosis_departamento=df_ambas_dosis_departamento.reset_index()
df_ambas_dosis_departamento=df_ambas_dosis_departamento.pivot(index='DEPARTAMENTO', columns='DOSIS', values='SEXO')
df_ambas_dosis_departamento.columns=['DOSIS1','DOSIS2']
df_ambas_dosis_departamento['POBLACION']=col_poblacion
df_ambas_dosis_departamento['INDICE']=round(df_ambas_dosis_departamento['DOSIS2']/(df_ambas_dosis_departamento['POBLACION']/100000)).astype('int')
df_ambas_dosis_departamento

,DOSIS1,DOSIS2,POBLACION,INDICE
DEPARTAMENTO,,,,
AMAZONAS,17836,5408,426806,1267
ANCASH,31651,17966,1180638,1522
APURIMAC,32575,7443,430736,1728
AREQUIPA,49573,32377,1497438,2162
AYACUCHO,38371,9311,668213,1393
CAJAMARCA,24896,14556,1453711,1001
CALLAO,102207,64590,1129854,5717
CUSCO,24862,13890,1357075,1024
HUANCAVELICA,27343,6162,365317,1687


In [6]:
# ACUMULADO POR GRUPO ETARIO (DOSIS 2 - VACUNACION COMPLETA)
bins = [18,30,40,50,60,70,80,df['EDAD'].max()+1]
labels = ['18 a 29 años','30 a 39 años','40 a 49 años','50 a 59 años','60 a 69 años','70 a 79 años','80 años a más']
poblacion_por_grupo_etario = [6303670,5031117,4183174,3277134,2221241,1271842,647355] 
df_edades = df
df_edades['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=False)
df_edades = df_edades[df_edades.DOSIS == 2].groupby(['GRUPO_ETARIO'])["DOSIS"].count().reset_index()
df_edades.rename(columns = {'DOSIS':'DOSIS2'}, inplace = True)
df_edades['POBLACION']=poblacion_por_grupo_etario
df_edades['PORCENTAJE']=round(df_edades['DOSIS2']/df_edades['POBLACION']*100,2)
df_edades=df_edades.set_index('GRUPO_ETARIO')
df_edades

,DOSIS2,POBLACION,PORCENTAJE
GRUPO_ETARIO,,,
18 a 19 años,10063,900000,1.12
20 a 29 años,126416,5700000,2.22
30 a 39 años,160768,5300000,3.03
40 a 49 años,119122,4400000,2.71
50 a 59 años,88114,3300000,2.67
60 a 69 años,55009,2300000,2.39
70 a 79 años,24225,1300000,1.86
80 años a más,59186,700000,8.46


In [7]:
#DIARIO POR FABRICANTE
df_fabricante=df[['FECHA_VACUNACION','FABRICANTE','SEXO']].groupby(['FECHA_VACUNACION','FABRICANTE']).count()
df_fabricante=df_fabricante.reset_index()
df_fabricante=df_fabricante.pivot(index='FECHA_VACUNACION', columns='FABRICANTE', values='SEXO')
df_fabricante=df_fabricante.rename_axis(None, axis=1)
df_fabricante=df_fabricante.fillna(0).astype('int')
df_fabricante

,ASTRAZENECA,PFIZER,SINOPHARM
FECHA_VACUNACION,,,
2021-02-09,0,0,3842
2021-02-10,0,0,16595
2021-02-11,0,0,33848
2021-02-12,0,0,32341
2021-02-13,0,0,18899
...,...,...,...
2021-05-05,8551,10216,2520
2021-05-06,7852,24932,405
2021-05-07,6770,26918,454


In [8]:
#ACUMULADO POR FABRICANTE
df_fabricante_cum=df_fabricante.cumsum()
df_fabricante_cum

,ASTRAZENECA,PFIZER,SINOPHARM
FECHA_VACUNACION,,,
2021-02-09,0,0,3842
2021-02-10,0,0,20437
2021-02-11,0,0,54285
2021-02-12,0,0,86626
2021-02-13,0,0,105525
...,...,...,...
2021-05-05,115246,933316,895715
2021-05-06,123098,958248,896120
2021-05-07,129868,985166,896574


In [30]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get('https://www.gob.pe/pongoelhombro#contador-de-vacunados')
scrap_contador_vacunados = r.html.find('.font-bold.text-3xl')
d = []
for number in scrap_contador_vacunados:
    d.append(number.text)
contador_vacunados = pd.Series(d,index=['total_dosis1', 'total_dosis2','ayer_dosis1','ayer_dosis2','total_dosis','ayer_total_dosis','vacunacion_completa'])
#contador_vacunados.to_json('../resultados/pongo_el_hombro.json')